In [1]:
# Importing pandas, numpy and TfidVectorizer
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.image as mpimg
import random
import networkx as nx
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel



In [2]:
# Reading data set from kaggle
df_sample_submission = pd.read_csv("sample_submission.csv")
df_article = pd.read_csv("articles.csv")
df_transaction_train = pd.read_csv("transactions_train.csv")
df_customer = pd.read_csv("customers.csv")

In [3]:
#Reading customer file for understanding of data
df_customer.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [4]:
#Reading article file for understanding of data

df_article.head()


,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [5]:
df_transaction_train.head()


,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [7]:
#Data preparation
item_freq = df_transaction_train.groupby('article_id')['customer_id'].nunique()
user_freq = df_transaction_train.groupby('customer_id')['article_id'].nunique()

items = item_freq[item_freq >= 100].index
users = user_freq[user_freq >= 100].index

filtered_df = df_transaction_train[df_transaction_train['article_id'].isin(items) & df_transaction_train['customer_id'].isin(users)]

In [8]:
freq = filtered_df.groupby(['customer_id', 'article_id']).size().reset_index(name='frequency')

GraphTravel_HM = filtered_df.merge(freq, on=['customer_id', 'article_id'], how='left')

GraphTravel_HM = GraphTravel_HM[GraphTravel_HM['frequency'] >= 10]

In [9]:
display(GraphTravel_HM)

print("unique customer_id" , GraphTravel_HM.customer_id.nunique())
print("unique article_id" , GraphTravel_HM.article_id.nunique())

,t_dat,customer_id,article_id,price,sales_channel_id,frequency
194,2018-09-20,06636eaa476a2f3417e8d11905e17a8066b4c9ae26bc78...,399201022,0.059305,2,12
195,2018-09-20,06636eaa476a2f3417e8d11905e17a8066b4c9ae26bc78...,399201022,0.059305,2,12
196,2018-09-20,06636eaa476a2f3417e8d11905e17a8066b4c9ae26bc78...,399201022,0.059305,2,12
1181,2018-09-20,1f58e8b3be25c4ca4bb457902f911cfbd4ed3a3f502685...,228257001,0.006763,2,19
1182,2018-09-20,1f58e8b3be25c4ca4bb457902f911cfbd4ed3a3f502685...,228257001,0.006797,2,19
...,...,...,...,...,...,...
6998469,2020-09-22,03fdb0bf2d9ff8ba23e1b4aef53709119aad5bc83691d8...,554450043,0.033881,2,18
6998949,2020-09-22,189a65461f77ed2d1621bb1201e68c4af1cfacf110d08e...,160442010,0.012898,2,11
6998950,2020-09-22,189a65461f77ed2d1621bb1201e68c4af1cfacf110d08e...,160442010,0.012915,2,11
6998951,2020-09-22,189a65461f77ed2d1621bb1201e68c4af1cfacf110d08e...,160442010,0.012898,2,11


unique customer_id 1013
unique article_id 922


In [10]:
# Create a dictionary to map unique customer IDs to indices
unique_customer_ids = GraphTravel_HM['customer_id'].unique()
customer_id_mapping = {id: index for index, id in enumerate(unique_customer_ids)}

# Update the 'customer_id' column in GraphTravel_HM using the mapping
GraphTravel_HM['customer_id'] = GraphTravel_HM['customer_id'].apply(lambda id: customer_id_mapping[id])

# Create a dictionary to map article IDs to product names
item_name_mapping = dict(zip(df_article['article_id'], df_article['prod_name']))


In [12]:
G = nx.Graph()

edges = [(row['customer_id'], row['article_id'], {'weight': row['frequency']})
         for _, row in GraphTravel_HM.iterrows()]

G.add_nodes_from([(id, {'type': 'user'}) for id in GraphTravel_HM['customer_id'].unique()])
G.add_nodes_from([(id, {'type': 'item'}) for id in GraphTravel_HM['article_id'].unique()])

G.add_edges_from(edges)


In [13]:
def biased_random_walk(G, start_node, walk_length, p=1, q=1):
    walk = [start_node]

    for _ in range(walk_length - 1):
        cur_node = walk[-1]
        cur_neighbors = list(G.neighbors(cur_node))

        if len(cur_neighbors) == 0:
            break

        prev_node = walk[-2] if len(walk) > 1 else None

        weights = [
            1 / p if neighbor == prev_node else
            1 if G.has_edge(neighbor, prev_node) else
            1 / q
            for neighbor in cur_neighbors
        ]

        next_node = random.choices(cur_neighbors, weights=weights, k=1)[0]
        walk.append(next_node)

    return walk


In [14]:
def generate_walks(G, num_walks, walk_length, p=1, q=1):
    walks = []  # List to store generated walks

    for _ in range(num_walks):
        shuffled_nodes = list(G.nodes())  # Get the list of nodes in the graph
        random.shuffle(shuffled_nodes)    # Shuffle nodes for randomness

        for node in shuffled_nodes:  # Iterate over shuffled nodes
            walk = biased_random_walk(G, node, walk_length, p, q)  # Generate a walk
            walks.append(walk)  # Add the generated walk to the list of walks

    return walks


In [17]:
# Generate Random Walks
walks = generate_walks(G, num_walks=10, walk_length=20, p=9, q=1)

# Filter out short walks
filtered_walks = [walk for walk in walks if len(walk) >= 5]

# Convert walks to string format (Word2Vec input)
string_walks = [[str(node) for node in walk] for walk in walks]

# Train Word2Vec model
model = Word2Vec(string_walks, vector_size=128, window=5, min_count=0, hs=1, sg=1, workers=4, epochs=10)

# Extract node embeddings
embeddings = {node_id: model.wv[node_id] for node_id in model.wv.index_to_key}


In [18]:
def get_user_embedding(user_id, embeddings):
    return embeddings[str(user_id)]

In [19]:
def get_rated_items(user_id, df):
    return set(df[df['customer_id'] == user_id]['article_id'])

Similarity Recommendation

In [23]:
def calculate_similarities(user_id, df, embeddings):
    rated_items = get_rated_items(user_id, df)
    user_embedding = get_user_embedding(user_id, embeddings).reshape(1, -1)  # Reshape to 2D array

    item_similarities = []
    for item_id in set(df['article_id']):
        if item_id not in rated_items:
            item_embedding = embeddings[str(item_id)].reshape(1, -1)  # Reshape to 2D array
            similarity = cosine_similarity(user_embedding, item_embedding)[0][0]
            item_similarities.append((item_id, similarity))

    return item_similarities


In [27]:
def recommend_items(user_id, df, embeddings, item_name_mapping, num_items=5):
    rated_items = get_rated_items(user_id, df)
    
    print(f"User {user_id} has purchased:")
    print('\n'.join([f"Item {item_id}: {item_name_mapping[item_id]}" for item_id in list(rated_items)[:5]]))
    
    item_similarities = calculate_similarities(user_id, df, embeddings)

    recommended_items = sorted(item_similarities, key=lambda x: x[1], reverse=True)[:num_items]

    print(f"\nRecommended items for user {user_id}:")
    print('\n'.join([f"Item {item_id}: {item_name_mapping[item_id]}, Similarity: {similarity:.4f}" for item_id, similarity in recommended_items]))


In [32]:
recommend_items(30, GraphTravel_HM, embeddings, item_name_mapping, num_items=5)

User 30 has purchased:
Item 665851001: Benny Slipper
Item 711053003: Tess Tee.
Item 189616006: Heavy jsy long leg
Item 477107015: Ellen Bikini Clover Mid 3p

Recommended items for user 30:
Item 784547001: Effie LS, Similarity: 0.7629
Item 824905002: Bendela LS, Similarity: 0.7337
Item 670435008: Billie, Similarity: 0.7331
Item 478298011: 2p jersey headband, Similarity: 0.7330
Item 633808016: Terrier tee TOP PRODUCT, Similarity: 0.7252


content based filtering

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# Load data from the CSV file
data = pd.read_csv("articles.csv")

In [7]:
# Drop rows with missing values in the 'combined_features' column
data = data.dropna(subset=['combined_features'])


In [8]:
# Sample user profile (replace with user's actual interactions)
user_profile = {
    'article_id': [108775015, 108775044]  # User has interacted with articles 108775015 and 108775044
}
user_profile_df = pd.DataFrame(user_profile)


In [9]:
# Combine product descriptions into a single text column
data['combined_features'] = data['prod_name'] + ' ' + data['detail_desc']


In [10]:
# Create TF-IDF vectors for product descriptions
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(data['combined_features'])


In [12]:
# Convert article_id to strings and filter the data
user_article_ids = user_profile_df['article_id'].astype(str)
filtered_data = data[data['article_id'].astype(str).isin(user_article_ids)]


In [13]:
# Create TF-IDF vectors for the user's profile
user_tfidf_matrix = tfidf_vectorizer.transform(filtered_data['combined_features'])


In [14]:
# Calculate cosine similarity between user profile and products
similarity_scores = cosine_similarity(user_tfidf_matrix, tfidf_matrix)


In [15]:
# Get indices of products sorted by similarity
recommended_indices = similarity_scores.argsort()[0][::-1]


In [24]:
# Get top N recommended product indices
N = 4
top_n_indices = recommended_indices[:N]


In [30]:
# Get actual product details for recommended products
recommended_products = data.loc[top_n_indices, ['article_id', 'prod_name', 'detail_desc']]
print(recommended_products)

       article_id      prod_name  \
0       108775015      Strap top   
1       108775044      Strap top   
2       108775051  Strap top (1)   
10053   538272001        Carmela   

                                             detail_desc  
0                Jersey top with narrow shoulder straps.  
1                Jersey top with narrow shoulder straps.  
2                Jersey top with narrow shoulder straps.  
10053  Blouse in a crêpe weave with a small stand-up ...  
